In [7]:
!pip install "ag2[retrievechat]==0.2.6" -qqq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## 예제 15.2 OpenAI API 키 설정

In [8]:
import json
import os
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

with open('OAI_CONFIG_LIST.json', 'w') as f:
  config_list = [
    {
        "model": "gpt-4.1-mini",
        "api_key": openai_api_key
    },
    {
        "model": "gpt-4o",
        "api_key": openai_api_key,
    },
    {
        "model": "dall-e-3",
        "api_key": openai_api_key,
    }
]
  json.dump(config_list, f)

## 예제 15.3 에이전트에 사용할 설정 불러오기

In [9]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST.json",
    file_location=".",
    filter_dict={
        "model": ["gpt-4.1-mini"],
    },
)

llm_config = {
    "config_list": config_list,
    "temperature": 0,
}

## 예제 15.4 AutoGen의 핵심 구성요소인 UserProxyAgent와 AssistantAgent

In [10]:
from autogen import AssistantAgent, UserProxyAgent

assistant = AssistantAgent("assistant", llm_config=llm_config)
user_proxy = UserProxyAgent("user_proxy",
  is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
  human_input_mode="NEVER",
  code_execution_config={"work_dir": "coding", "use_docker": False})

## 예제 15.5 삼성전자의 3개월 주식 가격 그래프를 그리는 작업 실행

In [11]:
user_proxy.initiate_chat(assistant, message="""
삼성전자의 지난 3개월 주식 가격 그래프를 그려서 samsung_stock_price.png 파일로 저장해줘.
plotly 라이브러리를 사용하고 그래프 아래를 투명한 녹색으로 채워줘.
값을 잘 확인할 수 있도록 y축은 구간 최소값에서 시작하도록 해줘.
이미지 비율은 보기 좋게 적절히 설정해줘.
""")

user_proxy (to assistant):


삼성전자의 지난 3개월 주식 가격 그래프를 그려서 samsung_stock_price.png 파일로 저장해줘.
plotly 라이브러리를 사용하고 그래프 아래를 투명한 녹색으로 채워줘.
값을 잘 확인할 수 있도록 y축은 구간 최소값에서 시작하도록 해줘.
이미지 비율은 보기 좋게 적절히 설정해줘.


--------------------------------------------------------------------------------


KeyboardInterrupt: 

## 예제 15.6 RAG 에이전트 클래스를 사용한 작업 실행

In [5]:
import autogen
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config=llm_config,
)

ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    retrieve_config={
        "task": "qa",
        "docs_path": "https://raw.githubusercontent.com/microsoft/autogen/main/README.md",
        "collection_name": "default-sentence-transformers"
    },
)

assistant.reset()
ragproxyagent.initiate_chat(assistant, problem="AutoGen이 뭐야?")

# assistant (to ragproxyagent):
# AutoGen은 여러 에이전트가 상호 대화하여 작업을 해결할 수 있는 LLM(Large Language Model) 애플리케이션 개발을 가능하게 하는 프레임워크입니다. AutoGen 에이전트는 사용자 정의 가능하며, 대화 가능하고, 인간 참여를 원활하게 허용합니다. LLM, 인간 입력, 도구의 조합을 사용하는 다양한 모드에서 작동할 수 있습니다.

Trying to create collection.


/home/restful3/anaconda3/envs/langgraph_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Number of requested results 20 is greater than number of elements in index 1, updating n_results = 1


doc_ids:  [['doc_0']]
Adding doc_id doc_0 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: AutoGen이 뭐야?

Context is: <a name="readme-top"></a>

<div align="center">
<img src="https://microsoft.github.io/autogen/0.2/img/ag.svg" alt="AutoGen Logo" width="100">

[![Twitter](https://img.shields.io/twitter/url/https/twitter.com/cloudposse.svg?style=social&label=Follow%20%40ag2)](https://twitter.com/ag2)
[![LinkedIn](https://img.shields.io/badge/LinkedIn-Company?style=flat&logo=linkedin&logoColor=white)](https://www.linkedin.com/company/105812540)
[![Discord](https://img.shields.io/badge/discord-chat-green?logo=discord)](https://aka.ms/autogen-discord)
[![Documentation](https://img.shields.io

## 예제 15.7 외부 정보를 활용하지 못하는 기본 에이전트의 답변

In [6]:
assistant.reset()
userproxyagent = autogen.UserProxyAgent(
    name="userproxyagent",
)
userproxyagent.initiate_chat(assistant, message="Autogen이 뭐야?")

# assistant (to userproxyagent):
# "Autogen"은 자동 생성을 의미하는 용어로, 주로 컴퓨터 프로그래밍에서 사용됩니다. 이는 코드, 문서, 또는 다른 데이터를 자동으로 생성하는 프로세스를 가리킵니다. 이는 반복적인 작업을 줄이고, 효율성을 높이며, 오류를 줄일 수 있습니다. 특정 컨텍스트에 따라 "Autogen"의 정확한 의미는 다를 수 있습니다.

userproxyagent (to assistant):

Autogen이 뭐야?

--------------------------------------------------------------------------------
assistant (to userproxyagent):

Autogen은 일반적으로 "자동 생성"을 의미하는 용어로, 다양한 분야에서 자동으로 무언가를 생성하는 기술이나 도구를 가리킵니다. 구체적으로 어떤 분야나 맥락에서 Autogen을 말씀하시는지에 따라 의미가 달라질 수 있습니다. 예를 들어:

1. **소프트웨어 개발**: 코드 자동 생성 도구를 의미할 수 있습니다. 예를 들어, API 클라이언트 코드, 데이터 모델, 설정 파일 등을 자동으로 생성해 주는 도구들이 있습니다.

2. **게임 개발**: 게임 내 맵, 캐릭터, 아이템 등을 자동으로 생성하는 시스템을 뜻할 수 있습니다.

3. **문서 작성**: 보고서나 문서의 일부를 자동으로 생성하는 기능을 의미할 수 있습니다.

4. **AI 및 머신러닝**: 텍스트, 이미지, 음성 등을 자동으로 생성하는 모델이나 시스템을 가리킬 수 있습니다.

만약 특정 제품명이나 서비스명으로서의 Autogen을 의미하신다면, 더 구체적인 정보를 알려주시면 자세히 설명해 드릴 수 있습니다.

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
userproxyagent (to assistant):



--------------------------------------------------------------------------------
assistant (to userproxyagent):

추가로 궁금한 점이나 더 알고 싶은 내용이 있으면 언제든지 말씀해 주세요! 도움이 필요하시면

KeyboardInterrupt: Interrupted by user

## 예제 15.8 OpenAI 임베딩 모델을 사용하도록 설정하기

In [ ]:
from chromadb.utils import embedding_functions

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=openai_api_key,
                model_name="text-embedding-3-small"
            )

ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    retrieve_config={
        "task": "qa",
        "docs_path": "https://raw.githubusercontent.com/microsoft/autogen/main/README.md",
        "embedding_function": openai_ef,
        "collection_name": "openai-embedding-3",
    },
)

assistant.reset()
ragproxyagent.initiate_chat(assistant, problem="Autogen이 뭐야?")

# assistant (to ragproxyagent):
# AutoGen은 여러 에이전트가 상호 대화하여 작업을 해결할 수 있는 LLM(Large Language Model) 애플리케이션 개발을 가능하게 하는 프레임워크입니다. AutoGen 에이전트는 사용자 정의 가능하며, 대화 가능하고, 인간 참여를 원활하게 허용합니다. LLM, 인간 입력, 도구의 조합을 사용하는 다양한 모드에서 작동할 수 있습니다.

## 예제 15.9 대화에 참여할 에이전트

In [ ]:
def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

# RAG를 사용하지 않는 사용자 역할 에이전트
user = autogen.UserProxyAgent(
    name="Admin",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    system_message="The boss who ask questions and give tasks.",
    code_execution_config=False,
    default_auto_reply="Reply `TERMINATE` if the task is done.",
)
# RAG를 사용하는 사용자 역할 에이전트
user_rag = RetrieveUserProxyAgent(
    name="Admin_RAG",
    is_termination_msg=termination_msg,
    system_message="Assistant who has extra content retrieval power for solving difficult problems.",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    code_execution_config=False,
    retrieve_config={
        "task": "code",
        "docs_path": "https://raw.githubusercontent.com/microsoft/autogen/main/samples/apps/autogen-studio/README.md",
        "chunk_token_size": 1000,
        "collection_name": "groupchat-rag",
    }
)
# 프로그래머 역할의 에이전트
coder = AssistantAgent(
    name="Senior_Python_Engineer",
    is_termination_msg=termination_msg,
    system_message="You are a senior python engineer. Reply `TERMINATE` in the end when everything is done.",
    llm_config=llm_config,
)
# 프로덕트 매니저 역할의 에이전트
pm = autogen.AssistantAgent(
    name="Product_Manager",
    is_termination_msg=termination_msg,
    system_message="You are a product manager. Reply `TERMINATE` in the end when everything is done.",
    llm_config=llm_config,
)

PROBLEM = "AutoGen Studio는 무엇이고 AutoGen Studio로 어떤 제품을 만들 수 있을까?"

## 예제 15.10 RAG 사용 여부에 따른 2개의 그룹챗 정의 및 실행

In [ ]:
def _reset_agents():
    user.reset()
    user_rag.reset()
    coder.reset()
    pm.reset()

def rag_chat():
    _reset_agents()
    groupchat = autogen.GroupChat(
        agents=[user_rag, coder, pm],
        messages=[], max_round=12, speaker_selection_method="round_robin"
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    user_rag.initiate_chat(
        manager,
        problem=PROBLEM,
    )

def norag_chat():
    _reset_agents()
    groupchat = autogen.GroupChat(
        agents=[user, coder, pm],
        messages=[],
        max_round=12,
        speaker_selection_method="auto",
        allow_repeat_speaker=False,
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    user.initiate_chat(
        manager,
        message=PROBLEM,
    )

## 예제 15.11 2개의 그룹챗을 실행한 결과 비교

In [ ]:
norag_chat()
# AutoGen Studio는 자동화된 코드 생성 도구입니다. 이 도구를 사용하면 개발자들이 더 빠르게, 더 효율적으로 코드를 작성할 수 있습니다.
# AutoGen Studio를 사용하면 다양한 유형의 소프트웨어 제품을 만들 수 있습니다. 예를 들어, 웹 애플리케이션, 모바일 애플리케이션, 데스크톱 애플리케이션, API, 데이터베이스 등을 만들 수 있습니다.
# ...

In [ ]:
rag_chat()
# AutoGen Studio는 AutoGen 프레임워크를 기반으로 한 AI 앱입니다. 이 앱은 AI 에이전트를 빠르게 프로토타입화하고, 스킬을 향상시키고, 워크플로우로 구성하고, 작업을 완료하기 위해 그들과 상호 작용하는 데 도움을 줍니다. 이 앱은 GitHub의 [microsoft/autogen](https://github.com/microsoft/autogen/tree/main/samples/apps/autogen-studio)에서 코드를 찾을 수 있습니다.
# AutoGen Studio를 사용하면 다음과 같은 기능을 수행할 수 있습니다:
# - 에이전트를 구축/구성하고, 그들의 구성(예: 스킬, 온도, 모델, 에이전트 시스템 메시지, 모델 등)을 수정하고, 워크플로우로 구성합니다.
# ...

## 예제 15.12 실습 준비하기

In [ ]:
import os
import re
import time
from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union

import matplotlib.pyplot as plt
import PIL
import requests
from openai import OpenAI
from PIL import Image

from autogen import Agent, AssistantAgent, ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.img_utils import _to_pil, get_image_data
from autogen.agentchat.contrib.multimodal_conversable_agent import MultimodalConversableAgent

config_list_4o = autogen.config_list_from_json(
    "OAI_CONFIG_LIST.json",
    filter_dict={
        "model": ["gpt-4o"],
    },
)

config_list_dalle = autogen.config_list_from_json(
    "OAI_CONFIG_LIST.json",
    filter_dict={
        "model": ["dall-e-3"],
    },
)

## 예제 15.13 DALLEAgent 정의

In [ ]:
def dalle_call(client, prompt, model="dall-e-3", size="1024x1024", quality="standard", n=1) -> str:
    response = client.images.generate(
        model=model,
        prompt=prompt,
        size=size,
        quality=quality,
        n=n,
    )
    image_url = response.data[0].url
    img_data = get_image_data(image_url)
    return img_data

class DALLEAgent(ConversableAgent):
    def __init__(self, name, llm_config: dict, **kwargs):
        super().__init__(name, llm_config=llm_config, **kwargs)

        try:
            config_list = llm_config["config_list"]
            api_key = config_list[0]["api_key"]
        except Exception as e:
            print("Unable to fetch API Key, because", e)
            api_key = os.getenv("OPENAI_API_KEY")
        self.client = OpenAI(api_key=api_key)
        self.register_reply([Agent, None], DALLEAgent.generate_dalle_reply)

    def generate_dalle_reply(self, messages, sender, config):
        client = self.client if config is None else config
        if client is None:
            return False, None
        if messages is None:
            messages = self._oai_messages[sender]

        prompt = messages[-1]["content"]
        img_data = dalle_call(client=self.client, prompt=prompt)
        plt.imshow(_to_pil(img_data))
        plt.axis("off")
        plt.show()
        return True, 'result.jpg'

## 예제 15.14 이미지 생성 에이전트 실행

In [ ]:
painter = DALLEAgent(name="Painter", llm_config={"config_list": config_list_dalle})

user_proxy = UserProxyAgent(
    name="User_proxy", system_message="A human admin.", human_input_mode="NEVER", max_consecutive_auto_reply=0
)

# 이미지 생성 작업 실행하기
user_proxy.initiate_chat(
    painter,
    message="갈색의 털을 가진 귀여운 강아지를 그려줘",
)

## 예제 15.15 이미지를 입력으로 받을 수 있는 GPT-4o 에이전트 생성

In [ ]:
image_agent = MultimodalConversableAgent(
    name="image-explainer",
    system_message="Explane input image for painter to create similar image.",
    max_consecutive_auto_reply=10,
    llm_config={"config_list": config_list_4o, "temperature": 0.5, "max_tokens": 1500},
)

user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=0,
    code_execution_config=False
)

groupchat = autogen.GroupChat(agents=[user_proxy, image_agent, painter], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

## 예제 15.16 유사한 이미지를 생성하도록 에이전트 실행

In [ ]:
user_proxy.initiate_chat(
    manager,
    message=f"""아래 이미지랑 비슷한 이미지를 만들어줘.
<img https://th.bing.com/th/id/R.422068ce8af4e15b0634fe2540adea7a?rik=y4OcXBE%2fqutDOw&pid=ImgRaw&r=0>.""",
)

## 예제 15.18 멀티 모달 에이전트에 텍스트로 명령

In [ ]:
user_proxy.initiate_chat(
    manager,
    message="갈색의 털을 가진 귀여운 강아지를 그려줘",
)